In [1]:
from MatrixFactor import *
from actions import *
from Graph import *
from expansions import *
from helpers import *
from Texable import render

In [6]:
x0 = N(0) * Trace(G(conjugated=True), B(1), G(), B(2))
render(x0)

for x in expand_G_loop(x0): #, lambda t: isinstance(t.factors[-1], DeltaKL)):
    render(x)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [3]:
x0 = N(0) * Trace(wtG(conjugated=True), B(1))
render(x0)

for x in expand_light_weight(x0): #, lambda t: isinstance(t.factors[-1], DeltaKL)):
    render(x)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>